In [10]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import os
import onnx
from qonnx.core.modelwrapper import ModelWrapper

file = "MaxPool_127.onnx"
fn = file.split('.')[0]
model = ModelWrapper("model_files/{}".format(file))
build_dir = "output_final/split_model_{}/custom_model".format(fn)

if not os.path.exists(build_dir):
    os.mkdir(build_dir)

### Tidy up

In [ ]:
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save(build_dir + "/end2end_{}_tidy.onnx".format(fn)

### Pre and postprocessing

In [ ]:
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType

model = ModelWrapper(build_dir+"/end2end_{}_tidy.onnx".format(fn))
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)